In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as transforms
import torchvision.models as models

from skimage import io, transform
import numpy as np
import pandas as pd
import os
from PIL import Image

In [2]:
NUM_EPOCHS = 10
LR = 1e-3
BATCH_SIZE = 4

In [3]:
tfms = transforms.Compose([transforms.Resize(256), transforms.RandomCrop(224), transforms.ToTensor()])

In [4]:
class AliensDataset(Dataset):

    def __init__(self, category_file, root_dir, num, transform=None):
        self.categories = pd.read_csv(os.path.join(root_dir,category_file))["category"]
        self.root_dir = root_dir
        self.transform = transform
        self.num = num

    def __len__(self):
        return len(self.categories)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir+"/training/", str(idx+1) + ".png")
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        category = self.categories[idx]
        sample = {'image': image, 'category': category}

        return sample

In [5]:
aliens_dataset = AliensDataset("solution.csv", "/home/krypt/myStuff/skillenza/paniithackathon2019/training.5k/training", 5000, tfms)

In [6]:
dataloader = DataLoader(aliens_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [7]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [8]:
model = models.resnet34(pretrained=True)

In [9]:
model = nn.Sequential(*list(model.children())[:-2],
                     nn.Conv2d(512, 6, 3, 1),
                     nn.AdaptiveAvgPool2d(1), Flatten(),
                     nn.Softmax())

if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [10]:
count = 0
for param in model.parameters():
    param.requires_grad = False
    count += 1
    if count >= 8:
        break

In [11]:
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    accuracy = 0
    count = 0
    for c, data in enumerate(dataloader):
        optimizer.zero_grad()

        images = data["image"]
        targets = data["category"].long()
        
        if torch.cuda.is_available():
            images = images.cuda()
            targets = targets.cuda()
        
        outs = model(images)
        loss = criterion(outs, targets)
        loss.backward()
        optimizer.step()

        
        total_loss += loss.data
        accuracy += torch.sum(torch.argmax(outs, dim=1) == (data["category"].cuda()-1))
        count += data["image"].size(0)

        if c%200 == 0:
            print("Epoch:", epoch, "Iter:", c, "Total loss:", total_loss, "Accuracy:", accuracy.cpu().numpy()/(count+1))
                
    print("Epoch:", epoch, "Total loss:", total_loss)
    print("Epoch:", epoch, "Accuracy:", accuracy/count)    

/home/krypt/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch: 0 Iter: 0 Total loss: tensor(1.8036, device='cuda:0') Accuracy: 0.0


RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/aten/src/THC/generic/THCTensorMathPairwise.cu:21